# Create Tilesheets for Bonus Phase

This creates latex figures that act as tilesheets for each of the planets.

First we load the planet textures from the source code.

In [2]:
import re
colors = {
    "00": "",
    "01": "\\BACKGROUNDONE",
    "10": "\\BACKGROUNDTWO",
    "11": "\\CHARCOLOR",
}
charsets_files = ["../iridisalpha/src/graphics/bonusphase_graphics.asm"]
charsets = {}
for charsets_file in charsets_files:
    input_file = open(charsets_file,'r')
    charset_data = []
    curr_charset = []
    for l in input_file.readlines():
        if "CHARACTER" in l or "fE" in l:
            if charset_data:
                curr_charset += [charset_data]
            charset_data = []
        if "fE" in l:
            if curr_charset:
                charsets[charset_name] = curr_charset
            charset_name = l.strip()
            curr_charset = []
            continue

        m = re.findall(r"[0-1]{8}",l)
        if not m:
            continue
        bits = m[0]
        for i in range(0,7,2):
            bitpair = bits[i:i+2]
            charset_data += [colors[bitpair]]
            charset_data += [colors[bitpair]]
    charsets[charset_name] = curr_charset


In [9]:
bp_charset = charsets["fE000"] + charsets["fE100"]

In [5]:
def fillCharacter(c, i=0, BIT_WIDTH=8):
    """
    Create a latex output that fills the cells in an 8x8 cell
    in the appropriate position in the grid.
    """
    output = ""
    gh = BIT_WIDTH - 1
    for j,v in enumerate(c):
        if not v:
            continue
        x = (i * BIT_WIDTH) + ((j % BIT_WIDTH) if j >= (BIT_WIDTH - 1) else j)
        y = (gh - int(j/BIT_WIDTH)) if j >= gh else gh
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    return output

In [11]:
planet_colors = {
    'fE000' : ['yellow','blue','red'],
    'fE100' : ['yellow','blue','red'],
    'Bonus_Phase' : ['yellow','blue','red'],
}

In [7]:
def createTile(planet_name, char, index, ref):
    """
    Create a tile in the tilesheet.
    """
    WIDTH = "0.5cm"
    header = f"""
    \\begin{{figure}}[H]
      {{
        \\setlength{{\\tabcolsep}}{{3.0pt}}
        \\setlength\cmidrulewidth{{\lightrulewidth}} % Make cmidrule = 
        \\begin{{adjustbox}}{{width={WIDTH},center}}
          \\begin{{tikzpicture}}
    """
    footer = f"""
          \end{{tikzpicture}}
        \end{{adjustbox}}
      }}\caption*{{\\{ref}}}
    \end{{figure}}
    """

    BIT_WIDTH = 8
    output = header + '\n'
    output += f"\\def\BACKGROUNDONE{{{planet_colors[planet_name][0]}}}\n"
    output += f"\\def\BACKGROUNDTWO{{{planet_colors[planet_name][1]}}}\n"
    output += f"\\def\CHARCOLOR{{{planet_colors[planet_name][2]}}}\n"
    
    grid = f"\\draw[step=1.0,gray,thin] (0,0) grid ({BIT_WIDTH},{BIT_WIDTH});"
    output += f"\t{grid}\n"

    output += fillCharacter(char)

    output += footer
    return output



Create a bunch of files of the form `planetCharset1_tilesheet.tex` that
contain a figure with the full tilesheet for that planet.

In [12]:
header = """
\\begin{figure}[H]                          
{                                          
  \\setlength{\\tabcolsep}{3.0pt}            
  \\setlength\cmidrulewidth{\lightrulewidth}
"""
figure = header

planet_name = "Bonus_Phase"
for i, c in enumerate(bp_charset):
    ref = f"${hex(0x40 + i)[2:].upper()}"
    output = createTile(planet_name, c, i, ref)
    of_name = f"../src/bonusphase/tilesheets/{planet_name}_tilesheet_{ref}.tex"
    with open(of_name,'w') as output_file:
        output_file.write(output)

    figure += "\\begin{subfigure}{0.12\\textwidth}\n"
    figure += f"\\input{{tilesheets/{planet_name}_tilesheet_{ref}}}\n"
    figure += "\\end{subfigure}\n"

footer = f"""
}}\\caption[]{{Tilesheet: Bonus Phase}}
\\end{{figure}}
"""
figure += footer
of_name = f"../src/bonusphase/{planet_name}_tilesheet.tex"
with open(of_name,'w') as output_file:
    output_file.write(figure)
